## Rasterize Point Data

In [1]:
import pandas as pd
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
import os
import rasterio
from rasterio import features
from rasterio.features import shapes
from rasterio.tools.mask import mask
from shapely.geometry import shape
import json
import cartopy.crs as ccrs
import xarray as xr
import georaster

## Setting up the Data and Variables

In [8]:
datapath=('C:\\Users\\Matt\\OneDrive\\Python\\Project\\clean_data')
g_datapath=('https://drive.google.com/drive/folders/15g7RWPRGI9-M31xGDDZhOm5rUXrK3Z02?usp=sharing/EAB.shp')

In [6]:
# Assign variable names for raster path/filenames
template_raster = (os.path.join(g_datapath,'s541.img'))
#ny_ash_out = (os.path.join(datapath,'NYs541.tif'))
output_raster = (os.path.join(g_datapath,'EAB_Proj.tif'))

In [7]:
# Open the shapefile as a geodataframe 
eab = gpd.read_file(os.path.join(g_datapath,'EAB.shp'))
eab_84 = eab.to_crs(epsg=4326)

OSError: no such file or directory: 'drive.google.com/drive/folders/15g7RWPRGI9-M31xGDDZhOm5rUXrK3Z02?usp=sharing\\EAB.shp'

In [ ]:
# Open the template raster file with rasterio
# Copy and update the metadata, setup transform and mask for the output
rst = rasterio.open(template_raster)
rst_meta = rst.meta.copy()
out_meta = rst.meta.copy()
#rst.window(*rst.bounds)

In [ ]:
# Open Census state boundaries as a geodataframe and select New York State
census_boundary = gpd.read_file(os.path.join(datapath, 'cb_2017_us_state_500k.shp'))
ny_boundary = census_boundary[census_boundary['STATEFP'] == "36"]
ny_boundary_84 = ny_boundary.to_crs(epsg=4326)
ny_boundary_proj = ny_boundary.to_crs(rst.crs.data)

In [ ]:
# Set coordinate system of shapefiles to same as raster, write out NY Boundary as shapefile to use with fiona
eab_proj=eab.to_crs(rst.crs.data)
#ny_boundary_proj=ny_boundary.to_crs(rst.crs.data)
ny_boundary_proj.to_file((os.path.join(datapath,'NY_Boundary.shp')))
#ny_bounds = ny_boundary_proj.total_bounds.tolist() # Use this to try clipping raster instead of shapefile/fiona geometry
#ny_boundary_utm=ny_boundary.to_crs(epsg=26918)
census_boundary_proj = census_boundary.to_crs(rst.crs.data)

## Plotting Vector Data

In [ ]:
ax=census_boundary.plot(figsize = (8,8), facecolor= 'white', edgecolor='black')
#ax.set_axis_off()
eab_84.plot(ax=ax, facecolor='red', edgecolor='black')
ax.set_ylim(20,50)
ax.set_xlim(-125,-60)

## "Zoom" in to NY

In [ ]:
ax=ny_boundary_84.plot(figsize=(8, 8), facecolor= 'white', edgecolor='black')
eab_84.plot(ax=ax, facecolor='red', edgecolor='black')
#ax.set_axis_off()

In [ ]:
# Get extent of New York boundary
with fiona.open((os.path.join(datapath,'NY_Boundary.shp'))) as shapefile:
    geoms = [feature["geometry"] for feature in shapefile]

## Plotting Vector with Raster 

In [ ]:
template_georaster = georaster.SingleBandRaster(template_raster)
with rasterio.open(template_raster) as src:
    crs = ccrs.AlbersEqualArea(central_longitude=-76)  
    fig = plt.figure(figsize=(8,8))
    ax = plt.subplot(projection=crs)
    plt.imshow(template_georaster.r, cmap = 'Purples', vmax=1, extent=template_georaster.extent)
    plt.xlim(-2500000,2500000)
    plt.ylim(300000,3200000)    
    census_boundary_proj.plot(ax=ax, facecolor='white', alpha = (0.3), edgecolor='black')
    ny_boundary_proj.plot(ax=ax, facecolor='yellow', alpha = (0.5), edgecolor='black')
    eab_proj.plot(ax=ax, facecolor = "red", alpha=(0.005))   

## "Zoom" in to NY 

In [ ]:
with rasterio.open(template_raster) as src:
    crs = ccrs.AlbersEqualArea(central_longitude=-76)  
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot(projection=crs)
    plt.imshow(template_georaster.r, cmap = 'Purples', vmax=1, extent=template_georaster.extent)
    plt.xlim(1300000,2000000)
    plt.ylim(2100000,2700000)
    ny_boundary_proj.plot(ax=ax, facecolor='yellow', alpha = (0.5), edgecolor='red')
    census_boundary_proj.plot(ax=ax, facecolor='white', alpha = (0.3), edgecolor='black')
    eab_proj.plot(ax=ax, facecolor='red', alpha=(0.05))    

## "Zoom" in to Buffalo Area

In [ ]:
with rasterio.open(template_raster) as src:
    crs = ccrs.AlbersEqualArea(central_longitude=-96)  
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot(projection=crs)
    plt.imshow(template_georaster.r, cmap = 'Purples', vmax=1, extent=template_georaster.extent)
    plt.xlim(1350000,1430000)
    plt.ylim(2300000,2380000)
    ny_boundary_proj.plot(ax=ax, facecolor='yellow', alpha = (0.5), edgecolor='black')
    eab_proj.plot(ax=ax, facecolor='red', alpha=(0.6))       

## "Zoom" in to see Pixels (250m x 250m)

In [ ]:
with rasterio.open(template_raster) as src:
    crs = ccrs.AlbersEqualArea(central_longitude=-96)  
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot(projection=crs)
    plt.imshow(template_georaster.r, cmap = 'Purples', vmax=1, extent=template_georaster.extent)
    plt.xlim(1375000,1385000)
    plt.ylim(2305000,2315000)
    ny_boundary_proj.plot(ax=ax, facecolor='yellow', alpha = (0.5), edgecolor='black')
    eab_proj.plot(ax=ax, facecolor='red', alpha=(0.6))    

## Output Template Raster (Clip/Extent)

In [ ]:
# Update the metadata, and setup transform and mask for the output
with rasterio.open(template_raster) as src:
    out_image, out_transform = mask(src, geoms, crop=True)
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                     "compress":'lzw'})    

In [ ]:
# Burn the features into the raster and write it out
with rasterio.open(output_raster, 'w', **out_meta) as out:
    out_arr = out.read(1) 
    shapes = ((geom,value) for geom, value in zip(eab_proj.geometry, eab_proj.OBJECTID))
    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out_transform)
    out.write_band(1, burned)

## New Variable names for Spatial Join

In [ ]:
# Assign variable names for raster path/filenames
raster_input = (os.path.join(datapath, 'EAB_Proj.tif'))
raster_output = (os.path.join(datapath, 'EAB_Proj_Count.tif'))

In [ ]:
rst = rasterio.open(raster_input)
t=rst.transform
out_meta = rst.meta.copy()
out_meta.update({"driver": "GTiff",
                  "compress": 'lzw'})

#out_meta.update({"crs": 26918, "driver": "GTiff",
#                  "compress": 'lzw'})

## Create Empty Array with same attributes as Template Raster

In [ ]:
rst_arr=rst.read(1)
rst_arr.shape

In [ ]:
boxes=np.arange(2032*2669).reshape(rst.shape).astype(np.int32)
plt.imshow(boxes)

## Make List of polygon shapes (for each cell) and zip to GeoJson 

In [ ]:
# Over 5 million rows! 
shapes=[]
values=[]

for shape, value in features.shapes(boxes, transform = t):
    shapes.append(shape)
    values.append(value)

In [ ]:
# Combine above values and shapes into GeoJson format
items=[]
for shape, value in zip(shapes, values):
    geo_json_format={}    
    geo_json_format['geometry']=shape
    geo_json_format['properties']={'value':value}
    items.append(geo_json_format)

In [ ]:
#Check if correct number of pixels
2032*2669==len(items)

##  Geodataframe from list, Spatial Join & Groupby

In [ ]:
# Create geodataframe from items, set projection and create unique id for each "cell" from the index. 
poly=gpd.GeoDataFrame.from_features(items)
poly.crs=eab_proj.crs
poly.reset_index(inplace=True)
poly.rename(columns={'index':'UID'}, inplace=True)
#poly.rename(columns={'OBJECTID':'pcount'}, inplace=True)
#poly.drop('value', 1, inplace=True)

In [ ]:
# This is the spatial join - it takes forever (at least 5 min on i7 4.0 GHz, 32 GB ram) and often crashes python with memory error
points_with_UID=gpd.sjoin(eab_proj, poly, how='left', op='within')

In [ ]:
# New geodataframe from groupby on unique id, count number of points (OBJECTID)
# Fill No Data with zeroes 
polys_w_values=poly.merge(pd.DataFrame(points_with_UID.groupby('UID')['OBJECTID'].count()), 
                        left_on='UID', right_index=True, how='left')
polys_w_values['OBJECTID']=polys_w_values.OBJECTID.fillna(0)
#polys_w_values.OBJECTID.max()#Check if it worked using max value

## Array from gdf, Output Raster with count of points per cell

In [ ]:
# Get geodataframe values into numpy array
val_array=np.array([val for val in polys_w_values.OBJECTID.tolist()])
array_for_raster=val_array.reshape(rst.shape)
array_for_raster=array_for_raster.astype(rasterio.float32)

In [ ]:
# Write the array values to the raster file
with rasterio.open(raster_output, 'w', **out_meta) as out:
    out.write(array_for_raster, 1)

## Plot Output

In [ ]:
#Read files with GeoRaster
template_raster = (os.path.join(datapath,'s541.img'))
output_raster = (os.path.join(datapath,'EAB_Proj_Count.tif'))
template_georaster = georaster.SingleBandRaster(template_raster)
extent_georaster = georaster.SingleBandRaster(output_raster)

In [ ]:
with rasterio.open(raster_output) as src:
    crs = ccrs.AlbersEqualArea(central_longitude=-96)  
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot(projection=crs)
    plt.imshow(template_georaster.r, cmap = 'Purples', vmax=1, extent=template_georaster.extent)
    plt.imshow(extent_georaster.r, alpha = 0.5, cmap = 'binary', vmax=1, extent=extent_georaster.extent)
    plt.xlim(1300000,2000000)
    plt.ylim(2100000,2700000)
    ny_boundary_proj.plot(ax=ax, facecolor='yellow', alpha = (0.5), edgecolor='red')
    census_boundary_proj.plot(ax=ax, facecolor='white', alpha = (0.3), edgecolor='black')

## "Zoom" in to see Pixels

In [ ]:
with rasterio.open(template_raster) as src:
    crs = ccrs.AlbersEqualArea(central_longitude=-96)  
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot(projection=crs)
    plt.imshow(extent_georaster.r, cmap = 'Purples', vmax=2, extent=extent_georaster.extent)
    plt.xlim(1375000,1385000)
    plt.ylim(2305000,2315000)
    ny_boundary_proj.plot(ax=ax, facecolor='yellow', alpha = (0.3), edgecolor='red')

## "Zoom" in to see Pixels with Original Point overlaid

In [ ]:
with rasterio.open(template_raster) as src:
    crs = ccrs.AlbersEqualArea(central_longitude=-96)  
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot(projection=crs)
    plt.xlim(1375000,1385000)
    plt.ylim(2305000,2315000)
    ny_boundary_proj.plot(ax=ax, facecolor='yellow', alpha = (0.3), edgecolor='red')
    plt.imshow(extent_georaster.r, cmap = 'Purples', vmax=2, extent=extent_georaster.extent)
    eab_proj.plot(ax=ax, facecolor='red', alpha=(0.6))    